# Content Based Recommendation

In [2]:
import os
import pandas as pd
import numpy as np

In [4]:
from IPython.display import Markdown, display
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## Data Preprocessing

In [5]:
DATA_DIR = os.path.abspath('./datasets/20_news')
OUTPUT_DIR = os.path.join('./datasets/output')

In [6]:
RAW_DATA_DIR = DATA_DIR
OUTPUT_DIR = os.path.join('./datasets/output')

In [7]:
classes = os.listdir(RAW_DATA_DIR)
columns=['id','class','text']

data_dict = {
    'id':[],
    'class':[],
    'text': []
}


In [8]:
for label in classes:
    files = os.path.join(RAW_DATA_DIR,label)
    for f in tqdm(os.listdir(files)):
        id_ = f.split('.')[0]
        try:
            with open(os.path.join(files,f),encoding='utf-8') as content:
                text = content.read().strip()
            #Adding corresponding data to the dictionary
            data_dict['id'].append(id_)
            data_dict['class'].append(label)
            data_dict['text'].append(text)
        except:
            print(label,id_,"Failed to load")


  0%|          | 0/200 [00:00<?, ?it/s]

Agriculture 101 Failed to load
Agriculture 102 Failed to load
Agriculture 103 Failed to load
Agriculture 104 Failed to load
Agriculture 105 Failed to load
Agriculture 106 Failed to load
Agriculture 107 Failed to load
Agriculture 108 Failed to load
Agriculture 109 Failed to load
Agriculture 110 Failed to load
Agriculture 111 Failed to load
Agriculture 112 Failed to load
Agriculture 113 Failed to load


  8%|▊         | 17/200 [00:00<00:01, 162.68it/s]

Agriculture 114 Failed to load
Agriculture 115 Failed to load
Agriculture 116 Failed to load
Agriculture 117 Failed to load
Agriculture 118 Failed to load
Agriculture 119 Failed to load
Agriculture 120 Failed to load
Agriculture 121 Failed to load
Agriculture 122 Failed to load
Agriculture 123 Failed to load
Agriculture 124 Failed to load
Agriculture 125 Failed to load
Agriculture 126 Failed to load


 16%|█▌        | 31/200 [00:00<00:01, 155.13it/s]

...

 67%|██████▋   | 211/313 [00:00<00:00, 343.05it/s]

World 290 Failed to load
World 291 Failed to load
World 292 Failed to load
World 293 Failed to load
World 294 Failed to load
World 295 Failed to load
World 296 Failed to load
World 297 Failed to load
World 298 Failed to load
World 299 Failed to load
World 3 Failed to load
World 300 Failed to load
World 301 Failed to load
World 302 Failed to load
World 303 Failed to load
World 304 Failed to load
World 305 Failed to load
World 306 Failed to load
World 307 Failed to load
World 308 Failed to load
World 309 Failed to load
World 310 Failed to load
World 311 Failed to load
World 312 Failed to load
World 313 Failed to load


 77%|███████▋  | 242/313 [00:00<00:00, 309.52it/s]

World 4 Failed to load
World 65 Failed to load
World 78 Failed to load


100%|██████████| 313/313 [00:00<00:00, 477.86it/s]


In [9]:
df = pd.DataFrame(data_dict,columns=columns)
df.columns
df

,id,class,text
0,1,Agriculture,﻿काठमाडौंमा पहिलो पटक स्ट्रबेरीको व्यवसायिक खे...
1,10,Agriculture,"जिल्लाका किसानले लगाएको अदुवामा गानो कुहिने, ग..."
2,100,Agriculture,काभ्रेपलाञ्चोकमा कृषकले एसआरआई प्रविधिमा गरेको...
3,11,Agriculture,राजधानीमा यतिबेला तरकारीको मूल्य आकासिएको छ। क...
4,12,Agriculture,पाल पोल्ट्री तथा लाइभस्टक क्षेत्रको समग्र विका...
5,13,Agriculture,माग अनुसारको आपूर्ति नभएपछि तरकारीको मुल्य दोब...
6,14,Agriculture,नेपाल र चीनबीचको सम्झौतासँगै शुरु भएको परियोजन...
7,15,Agriculture,यस वर्ष अम्बामा देशैभर अम्बामा रोग देखा पर्‍यो...
8,16,Agriculture,लैँचीको कारोबार ठप्प भएपछि चाडबाडका लागि खर्च ...
9,17,Agriculture,बदलिँदो मौसमका कारण मोरङमा उन्नत जातको धानबाली...


In [11]:
df.dropna(inplace=True) # drop rows with invalid values

## Processing data

In [13]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [14]:
nepali_stop_words = set(stopwords.words('nepali'))

tf = TfidfVectorizer(stop_words=nepali_stop_words,encoding='utf-8',decode_error='ignore')
tf = TfidfVectorizer(tokenizer= lambda x: x.split(" "),
                                  sublinear_tf=True, encoding='utf-8',
                                  decode_error='ignore',
                                  max_df=0.5,
                                  min_df=10,
                                  stop_words=nepali_stop_words)

tfidf_matrix = tf.fit_transform(df['text'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

E:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['कम', 'से'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
E:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


In [16]:
for idx, row in df.iterrows():
    idx = int(idx)
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df['id'][i], df['class'][i]) for i in similar_indices]

    results[row['id']] = similar_items[1:]
    
print('Completed!')
similar_indices = cosine_similarities[idx].argsort()[:-100:-1]

{'1': [(0.1796513911486855, '9', 'Tourism'), (0.17850697143653185, '201', 'Tourism'), (0.15114213314796357, '232', 'Tourism'), (0.14733313223927683, '294', 'Sports'), (0.1468411132855717, '431', 'Sports'), (0.13316390412527546, '12', 'Agriculture'), (0.13265956555680833, '362', 'Economy'), (0.13260563120144206, '339', 'Sports'), (0.12570261870736954, '602', 'Sports'), (0.12502534225044806, '645', 'Sports'), (0.12101796174172665, '622', 'Entertainment'), (0.11901326399774134, '99', 'Bank'), (0.11367704113257328, '619', 'Sports'), (0.11164532884021536, '183', 'Literature'), (0.11164532884021536, '230', 'Literature'), (0.11122264918732389, '458', 'Sports'), (0.11002525498146526, '435', 'Economy'), (0.10953182549869607, '171', 'Entertainment'), (0.10829944750388047, '347', 'Sports'), (0.10819690047394043, '551', 'Sports'), (0.10701183359186263, '41', 'Sports'), (0.105310927397082, '77', 'Business'), (0.105310927397082, '145', 'Interview'), (0.10484400436124697, '27', 'Sports'), (0.10357029


Completed!


## Displaying Results

In [40]:
def printBold(string):
    display(Markdown(string))

def item(id):
    id = str(id)
    return {'text': df.loc[df['id'] == id]['text'].tolist()[0].split(' - ')[0],
           'class': df.loc[df['id'] == id]['class'].tolist()[0].split(' - ')[0]
           }

def recommender(item_id, num): 
    print("\n")
    printBold("**Recommendation System**")
    print("The given news " + "'"+ item(item_id)['text'][:100] + "..." + "'" + " is of Category: " + str(item(item_id)['class'][:100]))
    
    print("\nThe " + str(num) + " most similar news in ascending order is given below: ")
    
    recs = results[str(item_id)][:num]
    for index, rec in enumerate(recs):
        print("\n")
        print(str(index+1) + ") " + "Category: " + str(rec[2]) + "\n\n" + "News: " + "'" + item(rec[1])['text'][:200] + "..." + "'" +" (score:" + str(rec[0]) + ")")

# Extract the news (that the user is currently interacting with) and provide 8 recommended news
recommender(item_id=120, num=10)

**Recommendation System**

The given news '﻿भुवन पौडेल
काठमाडौं, १२ माघ – नेपाल स्टक एक्सचेन्ज (नेप्से)ले आर्थिक वर्ष ०७१÷७२ मा २१ करोड ५० लाख ...' is of Category: Bank

The 10 most similar news in ascending order is given below: 


1) Category: Society

News: '﻿नेपालगन्ज, १५ असार । नेपालगन्ज वडा नं.२४ मा कार्यालय स्थापना गरेको उद्दमशीलता बचत तथा ऋण सहकारी संस्था लिमिटेडले बचतकर्ताको लाखौं रकम हिनामिना गरेको समाचार कारोबार दैनिकमा प्रकाशित भएपछि सो सहकारीले ...' (score:0.28849297877846714)


2) Category: Society

News: '﻿२६ चैत, काठमाडौं । आइएमई लिमिटेडले क्योदाई रेमिटेन्स जापानसँगको सहकार्यमा ल्याएको ‘घर जने हर्ष’ योजनाअन्तर्गत पहिलो साताको विजेता घोषणा गरेको छ ।
पहिलो साताको विजेतामा क्योटो जापानमा रहेका विजय श्रेष...' (score:0.24417191908850508)


3) Category: Entertainment

News: '﻿४ पुस, काठमाडौं । वातावरण मन्त्रालयलाई रोडा, ढुङ्गा र गिटीसम्बन्धी उद्योगको मापदण्ड निर्धारण गर्ने अख्तियारी दिएको छ ।

उद्योग मन्त्रालयमा  बसेको बैठकमा सो अख्तियारी दिएको हो ।
बैठकमा उद्योगमन्त्री न...' (score:0.23832787103008674